# Just Testing Run Vertex AI Pipeline Before Implement CI/CD

In [1]:
from google.cloud import aiplatform as vertex_ai

In [2]:
import os
import sys

SCRIPT_DIR = os.getcwd()

sys.path.append(os.path.normpath(os.path.join(SCRIPT_DIR, "..")))
import src.tfx_pipelines.config as config  # noqa: E402

In [16]:
# Note: to generate template, you need to run this first: python kubeflow_runner.py
sample_template_path = os.path.abspath(f"{config.PIPELINE_NAME}.json")
print(sample_template_path)

/home/hobiron/PlayML/tfdf-ft-pipeline/src/urge-classifier-train-pipeline.json


In [17]:
# We will upload the template into gcs
!gcloud config set project {config.GOOGLE_CLOUD_PROJECT}

Updated property [core/project].


In [18]:
!gsutil cp {sample_template_path} {config.GCS_PIPELINES_STORE}/

Copying file:///home/hobiron/PlayML/tfdf-ft-pipeline/src/urge-classifier-train-pipeline.json [Content-Type=application/json]...
- [1 files][ 22.9 KiB/ 22.9 KiB]                                                
Operation completed over 1 objects/22.9 KiB.                                     


In [10]:
gcs_compiled_pipeline_file_location = (
    f"{config.GCS_PIPELINES_STORE}/{config.PIPELINE_NAME}.json"
)

In [20]:
!gsutil cat {gcs_compiled_pipeline_file_location} | head

{"displayName": "urge-classifier-train-pipeline", "labels": {"tfx_py_version": "3-8", "tfx_runner": "kubeflow_v2", "tfx_version": "1-14-0"}, "pipelineSpec": {"components": {"BaselineModelResolver-model-blessing-resolver": {"executorLabel": "BaselineModelResolver-model-blessing-resolver_executor", "outputDefinitions": {"artifacts": {"model_blessing": {"artifactType": {"instanceSchema": "title: tfx.ModelBlessing\ntype: object\n"}}}}}, "BaselineModelResolver-model-resolver": {"executorLabel": "BaselineModelResolver-model-resolver_executor", "inputDefinitions": {"artifacts": {"input": {"artifactType": {"instanceSchema": "title: tfx.ModelBlessing\ntype: object\n"}}}}, "outputDefinitions": {"artifacts": {"model": {"artifactType": {"instanceSchema": "title: tfx.Model\ntype: object\n"}}}}}, "DataTransformer": {"executorLabel": "DataTransformer_executor", "inputDefinitions": {"artifacts": {"examples": {"artifactType": {"instanceSchema": "title: tfx.Examples\ntype: object\nproperties:\n  span:\n

In [19]:
job = vertex_ai.PipelineJob(
    display_name=config.PIPELINE_NAME, template_path=gcs_compiled_pipeline_file_location
)

In [21]:
job.submit()

Creating PipelineJob
PipelineJob created. Resource name: projects/677694625515/locations/us-central1/pipelineJobs/urge-classifier-train-pipeline-20240315022740
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/677694625515/locations/us-central1/pipelineJobs/urge-classifier-train-pipeline-20240315022740')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/urge-classifier-train-pipeline-20240315022740?project=677694625515
